Heart Disease Predictor
- Features: Sex, Age, ChestPainType, RestingBP, FastingBS, Exercise Angina
Dataset: heart_upd.csv

Algorithm: Stacked random-forest and logistic regression

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%pip install scipy
%pip install seaborn
import seaborn as sns
import sklearn
%matplotlib inline

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
dataset = pd.read_csv('../data/heart_upd.csv')
dataset

,Age,Sex,ChestPainType,RestingBP,FastingBS,ExerciseAngina,HeartDisease,Unnamed: 7,NewSex,NewExcercise,NewChestPain
0,40,M,ATA,140,0,N,0,NaN,1,0,2
1,49,F,NAP,160,0,N,1,NaN,0,0,1
2,37,M,ATA,130,0,N,0,NaN,1,0,2
3,48,F,ASY,138,0,Y,1,NaN,0,1,0
4,54,M,NAP,150,0,N,0,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,0,N,1,NaN,1,0,3
914,68,M,ASY,144,1,N,1,NaN,1,0,0
915,57,M,ASY,130,0,Y,1,NaN,1,1,0
916,57,F,ATA,130,0,N,1,NaN,0,0,2


In [7]:
Y = dataset['HeartDisease']
X = dataset.drop(['Unnamed: 7', 'Sex', 'ChestPainType', 'ExerciseAngina', 'HeartDisease'], axis=1)
X

,Age,RestingBP,FastingBS,NewSex,NewExcercise,NewChestPain
0,40,140,0,1,0,2
1,49,160,0,0,0,1
2,37,130,0,1,0,2
3,48,138,0,0,1,0
4,54,150,0,1,0,1
...,...,...,...,...,...,...
913,45,110,0,1,0,3
914,68,144,1,1,0,0
915,57,130,0,1,1,0
916,57,130,0,0,0,2


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .15, random_state = 0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [16]:
from sklearn.ensemble import RandomForestClassifier
rf_Model = RandomForestClassifier(n_estimators=150, max_depth=7, criterion="entropy")
rf_Model.fit(X_train, Y_train)

RandomForestClassifier(criterion='entropy', max_depth=7, n_estimators=150)

In [17]:
Y_pred = rf_Model.predict(X_test)

In [18]:
from sklearn.metrics import accuracy_score
a_score = accuracy_score(Y_test, Y_pred)
a_score

0.8115942028985508

In [19]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, Y_train)


LogisticRegression(random_state=0)

In [22]:
Y_pred_log = classifier.predict(X_test)

In [23]:
from sklearn.metrics import confusion_matrix
a_score_log = accuracy_score(Y_test, Y_pred_log)
a_score_log

0.8188405797101449

In [24]:
from sklearn.ensemble import StackingClassifier
estimators = [
    ('logistic_regression', classifier),
    ('random_forest', rf_Model)
]

In [29]:
from sklearn.neighbors import KNeighborsClassifier
stack_model = StackingClassifier(estimators=estimators, final_estimator=KNeighborsClassifier())
stack_model.fit(X_train, Y_train)

StackingClassifier(estimators=[('logistic_regression',
                                LogisticRegression(random_state=0)),
                               ('random_forest',
                                RandomForestClassifier(criterion='entropy',
                                                       max_depth=7,
                                                       n_estimators=150))],
                   final_estimator=KNeighborsClassifier())

In [30]:
y_pred_stacked = stack_model.predict(X_test)

In [31]:
a_score_stacked = accuracy_score(Y_test, y_pred_stacked)
a_score_stacked

0.7246376811594203